In [71]:
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas

from rasterio import windows
from itertools import product
import rasterio 
import gdal 
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/2018-07 La Paz/Site 5/lap_2018-07_site05_120m_RGB_cc.tif" .

In [106]:
import sys
sys.modules.keys()

dict_keys(['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'warnings', 're', 'enum', 'types', 'functools', '_functools', 'collections', '_collections_abc', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', '_collections', 'weakref', 'collections.abc', 'sre_compile', '_sre', 'sre_parse', 'sre_constants', 'copyreg', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'google', 'google.cloud', 'google.logging', 'mpl_toolkits', 'sphinxcontrib', 'sitec

In [129]:
import multiprocess
import threading
import concurrent
import numpy as np

In [124]:
'''
get_tiles implementation from 

https://gis.stackexchange.com/questions/285499/how-to-split-multiband-image-into-image-tiles-using-rasterio

'''
def load_image(file):
    img = rasterio.open(file)
    meta = img.meta.copy()
    return img, meta

def get_tiles(ds, width=256, height=256):
    out_window = []
    out_transform = []
    ncols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, ncols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=ncols, height=nrows)
    for col_off, row_off in  offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        out_window.append(window)
        out_transform.append(windows.transform(window, ds.transform))
    return out_window, out_transform



def generate_tiles(img, meta, files=False, width=256, height=256):

    out_path = 'images/'
    output_filename = 'tile_{}-{}.tif'
    window, transform = get_tiles(img, width, height)

    #locking read and write since they are not thread safe 
    read_lock = threading.Lock()
    write_lock = threading.Lock()

    def process(window,transform):
        with read_lock:
            tile = img.read(window=window)
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
        #checking for tiles that are only alpha band

        with write_lock:
            #if you want to write files 
            if files:
                outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(img.read(window=window))
        return tile
    results = []

    for cur_w, cur_t in zip(window, transform):
        with concurrent.futures.ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            future = executor.submit(process, cur_w, cur_t)
        results.append(future.result())
    return results

In [ ]:
# 16 seconds - intial implementation
# 13 seconds - gdal_retile.py
# 12.5 seconds - retile implementation
# 8.25 seconds - no file saving

In [126]:
#example usage of retile

%%time
file = "lap_2018-07_site05_120m_RGB_cc.tif" 
img, meta = load_image(file)
results = generate_tiles(img,meta,files=False)

CPU times: user 5.92 s, sys: 1.2 s, total: 7.12 s
Wall time: 8.24 s
